In [43]:
first_file_path = "../Fajlovi/1000G_test.vcf"
second_file_path = "../Fajlovi/1000G_test_2.vcf"

mergedFile = open("merged.vcf","w+")

class File_format: 
    def __init__(self, line):
        self.line = line
        self.vcf_version = ""
        self.extract_file_format_version()
        
    def __eq__(self, other):
        return self.line == other.line
    
    def extract_file_format_version(self):
        try:
            version = self.line.split("=VCFv",1)[1]
            self.vcf_version = version
        except:
            pass
    

class Filter:
    def __init__(self, line):
        self.line = line
        self.id = ""
        self.description = ""
        
    def extract_id(self):
        try:
            id = self.line.split("ID=", 1)[1]
            print("Id: " + id)
            self.id = id
        except:
            pass
    
    def extract_description(self):
        try:
            description = self.line.split('Description="', 1)[1]
            print("Desc: " + description)
            self.description = description
    

# context manager
with open(first_file_path) as first_file_object:
    for line in first_file_object:
        if(line.startswith("##fileformat")):
            file_format_first_file = File_format(line)
            
            with open(second_file_path) as second_file_object:
                first_line_in_second_file = second_file_object.readline()
                file_format_second_file = File_format(first_line_in_second_file) 
                if(file_format_first_file.vcf_version == file_format_second_file.vcf_version):
                    print("Isti format fajla je u oba VCF dokumenta")
                else:
                    print("Različit format fajlova u ova dva VCF dokumenta.")
                
            
        # ##FILTER=<ID="id",Description="description">    
        elif (line.startswith("##FILTER")):
                pass
        
    

Isti format fajla je u oba VCF dokumenta
